In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
!pip install datasets

In [3]:
from datasets import load_dataset

In [4]:
# from transformers import AutoModelForTokenClassification, AutoTokenizer


In [5]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import torch

tokenizer = AutoTokenizer.from_pretrained("procit008/stable_version_on_negative_and_conformation_NER_sep20")
model = AutoModelForTokenClassification.from_pretrained("procit008/stable_version_on_negative_and_conformation_NER_sep20")
dataset = load_dataset("conll2002", 'nl')
nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/15807 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2896 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5196 [00:00<?, ? examples/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'entity': 'LABEL_0', 'score': 0.9999981, 'index': 1, 'word': 'my', 'start': 0, 'end': 2}, {'entity': 'LABEL_0', 'score': 0.99999785, 'index': 2, 'word': 'name', 'start': 3, 'end': 7}, {'entity': 'LABEL_0', 'score': 0.9999982, 'index': 3, 'word': 'is', 'start': 8, 'end': 10}, {'entity': 'LABEL_1', 'score': 0.99997234, 'index': 4, 'word': 'wolfgang', 'start': 11, 'end': 19}, {'entity': 'LABEL_0', 'score': 0.99735284, 'index': 5, 'word': 'and', 'start': 20, 'end': 23}, {'entity': 'LABEL_0', 'score': 0.9999975, 'index': 6, 'word': 'i', 'start': 24, 'end': 25}, {'entity': 'LABEL_0', 'score': 0.99999094, 'index': 7, 'word': 'live', 'start': 26, 'end': 30}, {'entity': 'LABEL_0', 'score': 0.99999404, 'index': 8, 'word': 'in', 'start': 31, 'end': 33}, {'entity': 'LABEL_5', 'score': 0.9999509, 'index': 9, 'word': 'berlin', 'start': 34, 'end': 40}]


In [11]:
def aggregate_word_level_predictions(tokenized_sentence, predictions, dataset):
    # print("tokenized_sentence",tokenized_sentence)
    # print("predictions",predictions)
    aggregated_predictions = []
    current_word = ""
    current_entity = ""

    for token, pred in zip(tokenized_sentence, predictions):
        if token.startswith("##"):
            # print("token",token)
            current_word += token[2:]  # Append sub-token to current word
            # print("current_word",current_word)
        else:
            if current_word:  # If there's a current word, add it with its entity
                aggregated_predictions.append((current_word, current_entity))
                # print("aggregated_predictions",aggregated_predictions)
            # Update the current word and entity
            current_word = token if token not in ["[CLS]", "[SEP]"] else ""
            # print("current_word",current_word)
            current_entity = dataset['train'].features['ner_tags'].feature.int2str(pred.item())
            # print("current_entity",current_entity)

    # Add the last word
    if current_word:
        aggregated_predictions.append((current_word, current_entity))

    return aggregated_predictions

In [12]:
def analyze(sentence):
    with torch.no_grad():
        inputs = tokenizer(sentence, return_tensors="pt")
        # print("analyze_input",inputs)
        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
        # print("tokens",tokens)
        outputs = model(**inputs).logits
        # print("outputs",outputs)
        predictions = torch.argmax(outputs, dim=2)
        # print("predictions",predictions)
        word_predictions = aggregate_word_level_predictions(tokens, predictions[0], dataset)
        print("word_predictions:", word_predictions, "\n")

        return {word: entity for word, entity in word_predictions}

In [19]:
text="no my name is aagtje"
analyze(text)

word_predictions: [('no', 'O'), ('my', 'O'), ('name', 'O'), ('is', 'O'), ('aagtje', 'B-PER')] 



{'no': 'O', 'my': 'O', 'name': 'O', 'is': 'O', 'aagtje': 'B-PER'}

In [14]:
texts = [
 "no",
 "nope",
 "not",
 "won't",
 "will not",
 "can't",
 "cannot",
 "negative",
 "nay",
 "absolutely not",
 "certainly not",
 "never",
 "no way",
 "not at all",
 "definitely not",
 "under no circumstances",
 "decline",
 "refuse",
 "disagree",
 "not going to",
 "don't",
 "do not",
 "sorry",
 "no",
 "not",
 "prefer not",
 "definitely not",
 "never",
 "unfortunately",
 "sadly",
 "regrettably",
 "not really",
 "not particularly",
 "not good",
 "not nice",
 "not fantastic",
 "not great",
 "not top",
 "not brilliant",
 "not super",
 "not wonderful",
 "not splendid",
 "not perfect",
 "not outstanding",
 "not excellent",
 "not phenomenal",
 "not fabulous",
 "not obvious",
 "not without doubt",
 "not wholeheartedly",
 "not with pleasure",
 "definitely",
 "not",
 "certainly not",
 "absolutely no",
 "really not",
 "not good",
 "not okay",
 "not happy",
 "not happy with",
 "disappointing",
 "bad",
 "less good",
 "terrible",
 "unfavorable",
 "unpleasant",
 "not positive",
 "not satisfied",
 "not fulfilled",
 "not satisfactory",
 "not pleasant",
 "not desirable",
 "not welcome",
 "not appreciated",
 "not worth it",
 "i do not",
 "do not",
 "donot",
 "nee dat klopt niet",
 "nee dat",
 "nee dat klopt",
 "nee",
 "nee",
 "nee",
 "niet",
 "jammer",
 "helaas",
 "spijtig",
 "absoluut niet",
 "zeker niet",
 "natuurlijk niet",
 "onmogelijk",
 "helaas wel",
 "absoluut geen optie",
 "ik ben er niet blij mee",
 "dat werkt niet",
 "verschrikkelijk",
 "vreselijk",
 "slecht",
 "waardeloos",
 "dramatisch",
 "rampzalig",
 "ellendig",
 "abominabel",
 "afschuwelijk",
 "onaangenaam",
 "niet goed",
 "negatief",
 "ontevreden",
 "frustrerend",
 "teleurstellend",
 "onzin",
 "belabberd",
 "bagger",
 "rotzooi",
 "klote",
 "vreselijke tegenvaller",
 "niet om over naar huis te schrijven",
 "dat kan beter",
 "ik ben er op tegen",
 "niet akkoord",
 "no youre mistaken",
 "no thats not correct",
 "no thats wrong",
 "no im afraid thats not true",
 "no thats inaccurate",
 "no thats not my name",
 "no youve got it wrong",
 "no thats not the case",
 "no thats false",
 "no thats completely incorrect",
 "no thats not right",
 "no youve misunderstood",
 "no im sorry but thats wrong",
 "no thats not the answer",
 "no youre off the mark",
 "no thats incorrect",
 "no i dont think thats right",
 "no thats not how it is",
 "no youve got the wrong idea",
 "no i cant agree with that",
 "no thats not quite right",
 "no youre off base with that",
 "no im afraid youre wrong",
 "no thats not the truth",
 "no thats a mistake",
 "no thats not how it goes",
 "no youve got the facts wrong",
 "no thats not the right information",
 "no i cant say thats true",
 "no thats not even close",
 "no thats far from correct",
 "no thats not what it is",
 "no youve been misinformed",
 "no thats off track",
 "no thats simply wrong",
 "no thats a misunderstanding",
 "no im afraid thats incorrect",
 "no you didnt get it right",
 "no thats not accurate",
 "no youve missed the point",
 "no youre mistaken about that",
 "no thats not exactly true",
 "no youve got it twisted",
 "no thats not what i said",
 "no i dont think thats right",
 "no thats totally off",
 "no youve misunderstood the situation",
 "no thats far from the truth",
 "no youve got the wrong end of stick",
 "no thats a misinterpretation",
 "no im afraid thats off",
 "no thats definitely not correct",
 "no youve got the wrong idea there",
 "no thats not what i meant",
 "no thats completely false",
 "no im afraid youve got that wrong",
 "no thats not true at all",
 "no thats not accurate information",
 "no youre way off base",
 "no thats far from right",
 "no thats totally incorrect",
 "no thats not how things are",
 "no youve gotten the wrong facts",
 "no thats not the correct assumption",
 "no youve made a mistake",
 "no thats not quite accurate",
 "no i cant agree with that at all",
 "no youre completely wrong",
 "no i dont believe thats correct",
 "no youve misunderstood my point",
 "no youre off target",
 "no thats not what happened",
 "no thats not how works",
 "no youve got it backwards",
 "no thats complete misunderstanding",
 "no thats not the situation",
 "no i dont think thats right",
 "no i cant agree with that perspective",
 "no youve got the wrong interpretation",
 "no youre mistaken in your conclusion",
 "no thats misconception",
 "no youre not understanding correctly",
 "no thats completely off base",
 "no thats definitely not the truth",
 "no youve misunderstood the facts",
 "no thats false statement",
 "no thats not the reality",
 "no thats not a true statement",
 "no youre entirely wrong",
 "no i cant say thats true at all",
 "no thats totally incorrect information",
 "no thats not full picture",
 "no thats not how things stand",
 "no thats even remotely true",
 "no thats misreading of the facts",
 "no thats an inaccurate representation",
 "no thats far from correct",
 "no thats not even close to the truth",
 "no thats an utterly false statement",
 "no thats entirely incorrect",
 "no thats an absolute misinterpretation",
 "no thats clearly not the case",
 "no thats completely misunderstanding",
 "no thats a gross misinterpretation",
 "no thats a total misunderstanding",
 "no thats a misreading",
 "no thats a blatant inaccuracy",
 "no thats an incorrect assertion",
 "no thats a basic mistake",
 "no thats a serious error",
 "no thats a huge misunderstanding",
 "no thats a serious misunderstanding",
 "no thats a blatant error",
 "no thats a serious misconception",
 "no thats a major misunderstanding",
 "no thats a major mistake",
 "no thats a major error",
 "no thats a serious misinterpretation",
 "no thats a gross misunderstanding",
 "no thats a blatant misinterpretation",
 "no thats a clear mistake",
 "no thats a major blunder",
 "no thats a simple error",
 "no thats a serious mistake",
 "no thats a clear misinterpretation",
 "no thats a serious misunderstanding",
 "no thats a complete misunderstanding",
 "no thats a huge mistake",
 "no thats a major misunderstanding",
 "no thats a grave error",
 "no thats a serious error",
 "no thats a major blunder",
 "no thats a big mistake",
 "no thats a significant error",
 "no thats a grave mistake",
 "no thats a serious misjudgment",
 "no thats a significant misunderstanding",
 "no thats a gross mistake",
 "no thats a gross error",
 "no thats a major misjudgment",
 "no thats a serious misstep",
 "no thats a serious blunder",
 "no thats a serious error",
 "no thats a gross misunderstanding",
 "no thats a huge mistake",
 "no thats a grave misinterpretation",
 "no thats a significant misinterpretation",
 "nee je vergist",
 "nee dat niet correct",
 "nee dat onjuist",
 "nee ik ben bang dat niet waar",
 "nee dat onjuist",
 "nee dat niet mijn naam",
 "nee je hebt het verkeerd",
 "nee dat niet de zaak",
 "nee dat onwaar",
 "nee dat helemaal onjuist",
 "nee dat niet juist",
 "nee je hebt misbegrepen",
 "nee ik ben sorry maar dat verkeerd",
 "nee dat niet het antwoord",
 "nee dat onjuist",
 "nee ik denk dat niet juist",
 "nee dat niet hoe het is",
 "nee je hebt het verkeerd idee",
 "nee ik kan het niet mee eens",
 "nee dat niet helemaal juist",
 "nee ik ben bang dat je verkeerd",
 "nee dat niet de waarheid",
 "nee dat een fout",
 "nee dat niet hoe het gaat",
 "nee je hebt de feiten verkeerd",
 "nee dat niet de juiste informatie",
 "nee ik kan niet zeggen dat waar",
 "nee dat niet zelfs dichtbij",
 "nee dat niet wat het is",
 "nee je hebt onjuiste informatie",
 "nee dat ver",
 "nope thats not true",
 "nope thats not correct",
 "nope thats a mistake",
 "nope thats an error",
 "nope thats a misunderstanding",
 "nope thats a blunder",
 "nope thats a false statement",
 "nope thats an incorrect assertion",
 "nope thats a gross mistake",
 "nope thats a serious error",
 "nope thats a huge misunderstanding",
 "nope thats a major mistake",
 "nope thats a grave error",
 "nope thats a clear mistake",
 "nope thats a simple error",
 "nope thats a blatant error",
 "nope thats a significant mistake",
 "nope thats a gross error",
 "nope thats a major blunder",
 "nope thats a blatant misinterpretation",
 "nope thats a major misjudgment",
 "nope thats a serious misinterpretation",
 "nope thats a major misunderstanding",
 "nope thats a serious blunder",
 "nope thats a gross misinterpretation",
 "nope thats a complete misunderstanding",
 "nope thats a huge mistake",
 "nope thats a grave misinterpretation",
 "nope thats a significant misinterpretation",
 "nee dat niet waar",
 "nee dat niet correct",
 "nee dat een fout",
 "nee dat een fout",
 "nee dat een misverstand",
 "nee dat een blunder",
 "nee dat een valse stelling",
 "nee dat een onjuiste beweerd",
 "nee dat een grote fout",
 "nee dat een serieuze fout",
 "nee dat een grote misverstand",
 "nee dat een belangrijke fout",
 "nee dat een ernstige fout",
 "nee dat een duidelijke fout",
 "nee dat een simpele fout",
 "nee dat een flagrante fout",
 "nee dat een significante fout",
 "nee dat een grote fout",
 "nee dat een belangrijke blunder",
 "nee dat een flagrante misinterpretatie",
 "nee dat een grote misjudgment",
 "nee dat een serieuze misinterpretatie",
 "nee dat een grote misverstand",
 "nee dat een serieuze blunder",
 "nee dat een grote misinterpretatie",
 "nee dat een volledige misverstand",
 "nee dat een grote fout",
 "nee dat een grave misinterpretatie",
 "nee dat een significante misinterpretatie",
 "nay thats not true",
 "nay thats not correct",
 "nay thats a mistake",
 "nay thats an error",
 "nay thats a misunderstanding",
 "nay thats a blunder",
 "nay thats a false statement",
 "nay thats an incorrect assertion",
 "nay thats a gross mistake",
 "nay thats a serious error",
 "nay thats a huge misunderstanding",
 "nay thats a major mistake",
 "nay thats a grave error",
 "nay thats a clear mistake",
 "nay thats a simple error",
 "nay thats a blatant error",
 "nay thats a significant mistake",
 "nay thats a gross error",
 "nay thats a major blunder",
 "nay thats a blatant misinterpretation",
 "nay thats a major misjudgment",
 "nay thats a serious misinterpretation",
 "nay thats a major misunderstanding",
 "nay thats a serious blunder",
 "nay thats a gross misinterpretation",
 "nay thats a complete misunderstanding",
 "nay thats a huge mistake",
 "nay thats a grave misinterpretation",
 "nay thats a significant misinterpretation",
 "absolutely not true",
 "absolutely not correct",
 "absolutely not my name",
 "absolutely not the case",
 "absolutely not false",
 "absolutely not completely incorrect",
 "absolutely not right",
 "absolutely not misunderstood",
 "absolutely not the answer",
 "absolutely not off the mark",
 "absolutely not i dont think",
 "absolutely not the right idea",
 "absolutely not quite right",
 "absolutely not off base",
 "absolutely not the truth",
 "absolutely not a mistake",
 "absolutely not how it goes",
 "absolutely not the facts wrong",
 "absolutely not the right information",
 "absolutely not even close",
 "absolutely not far from correct",
 "absolutely not youve been misinformed",
 "absolutely not off track",
 "absolutely not simply wrong",
 "absolutely not a misunderstanding",
 "absolutely not you didnt get it right",
 "absolutely not accurate",
 "absolutely not missed the point",
 "absolutely not exactly true",
 "absolutely not twisted",
 "absolutely not what i said",
 "absolutely not totally off",
 "absolutely not the whole thing",
 "absolutely not a misreading of the facts",
 "absolutely not aligned with the truth",
 "absolutely not i have to disagree",
 "absolutely not correct interpretation",
 "absolutely not the right answer",
 "absolutely not the wrong",
 "absolutely not something is amiss",
 "absoluut niet waar",
 "absoluut niet juist",
 "absoluut niet mijn naam",
 "absoluut niet de zaak",
 "absoluut niet vals",
 "absoluut niet volledig onjuist",
 "absoluut niet juist",
 "absoluut niet misbegrepen",
 "absoluut niet de antwoord",
 "absoluut niet ver de mark",
 "absoluut niet ik denk",
 "absoluut niet de juiste idee",
 "absoluut niet helemaal juist",
 "absoluut niet ver basis",
 "absoluut niet de waarheid",
 "absoluut niet een fout",
 "absoluut niet hoe het gaat",
 "absoluut niet de feiten verkeerd",
 "absoluut niet de juiste informatie",
 "absoluut niet zelfs dichtbij",
 "absoluut niet ver van juist",
 "absoluut niet je bent misleid",
 "absoluut niet van de spoor",
 "absoluut niet simpelweg verkeerd",
 "absoluut niet een misverstand",
 "absoluut niet je hebt het juist",
 "absoluut niet nauwkeurig",
 "absoluut niet gemist de punt",
 "absoluut niet precies waar",
 "absoluut niet vervormd",
 "absoluut niet wat ik zei",
 "absoluut niet totaal ver",
 "absoluut niet de hele zaak",
 "absoluut niet een foute lezen van de feiten",
 "absoluut niet in lijn met de waarheid",
 "absoluut niet ik moet het oneens",
 "absoluut niet juiste interpretatie",
 "absoluut niet de juiste antwoord",
 "absoluut niet de verkeerd",
 "absoluut niet iets is mis",
 "won't thats not true",
 "won't thats not correct",
 "won't thats a mistake",
 "won't thats an error",
 "won't thats a misunderstanding",
 "won't thats a blunder",
 "won't thats a false statement",
 "won't thats an incorrect assertion",
 "won't thats a gross mistake",
 "won't thats a serious error",
 "won't thats a huge misunderstanding",
 "won't thats a major mistake",
 "won't thats a grave error",
 "won't thats a clear mistake",
 "won't thats a simple error",
 "won't thats a blatant error",
 "won't thats a significant mistake",
 "won't thats a gross error",
 "won't thats a major blunder",
 "won't thats a blatant misinterpretation",
 "won't thats a major misjudgment",
 "won't thats a serious misinterpretation",
 "won't thats a major misunderstanding",
 "won't thats a serious blunder",
 "won't thats a gross misinterpretation",
 "won't thats a complete misunderstanding",
 "won't thats a huge mistake",
 "won't thats a grave misinterpretation",
 "won't thats a significant misinterpretation",
 "certainly thats not true",
 "certainly thats not correct",
 "certainly thats a mistake",
 "certainly thats an error",
 "certainly thats a misunderstanding",
 "certainly thats a blunder",
 "certainly thats a false statement",
 "certainly thats an incorrect assertion",
 "certainly thats a gross mistake",
 "certainly thats a serious error",
 "certainly thats a huge misunderstanding",
 "certainly thats a major mistake",
 "certainly thats a grave error",
 "certainly thats a clear mistake",
 "certainly thats a simple error",
 "certainly thats a blatant error",
 "certainly thats a significant mistake",
 "certainly thats a gross error",
 "certainly thats a major blunder",
 "certainly thats a blatant misinterpretation",
 "certainly thats a major misjudgment",
 "certainly thats a serious misinterpretation",
 "certainly thats a major misunderstanding",
 "certainly thats a serious blunder",
 "certainly thats a gross misinterpretation",
 "certainly thats a complete misunderstanding",
 "certainly thats a huge mistake",
 "certainly thats a grave misinterpretation",
 "certainly thats a significant misinterpretation",
 "zeker dat niet waar",
 "zeker dat niet juist",
 "zeker dat een fout",
 "zeker dat een fout",
 "zeker dat een misverstand",
 "zeker dat een blunder",
 "zeker dat een valse stelling",
 "zeker dat een onjuist stelling",
 "zeker dat een grove fout",
 "zeker dat een ernstige fout",
 "zeker dat een enorm misverstand",
 "zeker dat een groot fout",
 "zeker dat een ernstig fout",
 "zeker dat een duidelijk fout",
 "zeker dat een simpel fout",
 "zeker dat een flagrante fout",
 "zeker dat een significant fout",
 "zeker dat een grove fout",
 "zeker dat een groot blunder",
 "zeker dat een flagrante misinterpretatie",
 "zeker dat een groot misjudgement",
 "zeker dat een ernstig misinterpretatie",
 "zeker dat een groot misverstand",
 "zeker dat een ernstig blunder",
 "zeker dat een grove misinterpretatie",
 "zeker dat een volledig misverstand",
 "zeker dat een enorm fout",
 "zeker dat een ernstig misinterpretatie",
 "zeker dat een significant misinterpretatie",
 "never thats not true",
 "never thats not correct",
 "never thats a mistake",
 "never thats an error",
 "never thats a misunderstanding",
 "never thats a blunder",
 "never thats a false statement",
 "never thats an incorrect assertion",
 "never thats a gross mistake",
 "never thats a serious error",
 "never thats a huge misunderstanding",
 "never thats a major mistake",
 "never thats a grave error",
 "never thats a clear mistake",
 "never thats a simple error",
 "never thats a blatant error",
 "never thats a significant mistake",
 "never thats a gross error",
 "never thats a major blunder",
 "never thats a blatant misinterpretation",
 "never thats a major misjudgment",
 "never thats a serious misinterpretation",
 "never thats a major misunderstanding",
 "never thats a serious blunder",
 "never thats a gross misinterpretation",
 "never thats a complete misunderstanding",
 "never thats a huge mistake",
 "never thats a grave misinterpretation",
 "never thats a significant misinterpretation",
 "nooit dat niet waar",
 "nooit dat niet juist",
 "nooit dat een fout",
 "nooit dat een fout",
 "nooit dat een misverstand",
 "nooit dat een blunder",
 "nooit dat een valse stelling",
 "nooit dat een onjuist stelling",
 "nooit dat een grove fout",
 "nooit dat een ernstig fout",
 "nooit dat een enorm misverstand",
 "nooit dat een groot fout",
 "nooit dat een ernstig fout",
 "nooit dat een helder fout",
 "nooit dat een eenvoudig fout",
 "nooit dat een flagrante fout",
 "nooit dat een belangrijk fout",
 "nooit dat een grove fout",
 "nooit dat een groot blunder",
 "nooit dat een flagrante misinterpretatie",
 "nooit dat een groot misbeoordeling",
 "nooit dat een ernstig misinterpretatie",
 "nooit dat een groot misverstand",
 "nooit dat een ernstig blunder",
 "nooit dat een grove misinterpretatie",
 "nooit dat een volledig misverstand",
 "nooit dat een enorm fout",
 "nooit dat een ernstig misinterpretatie",
 "nooit dat een belangrijk misinterpretatie",
 "at all not true",
 "at all not correct",
 "at all a mistake",
 "at all an error",
 "at all a misunderstanding",
 "at all a blunder",
 "at all a false statement",
 "at all an incorrect assertion",
 "at all a gross mistake",
 "at all a serious error",
 "at all a huge misunderstanding",
 "at all a major mistake",
 "at all a grave error",
 "at all a clear mistake",
 "at all a simple error",
 "at all a blatant error",
 "at all a significant mistake",
 "at all a gross error",
 "at all a major blunder",
 "at all a blatant misinterpretation",
 "at all a major misjudgment",
 "at all a serious misinterpretation",
 "at all a major misunderstanding",
 "at all a serious blunder",
 "at all a gross misinterpretation",
 "at all a complete misunderstanding",
 "at all a huge mistake",
 "at all a grave misinterpretation",
 "at all a significant misinterpretation",
 "bij alle niet waar",
 "bij alle niet correct",
 "bij alle een fout",
 "bij alle een fout",
 "bij alle een misverstand",
 "bij alle een blunder",
 "bij alle een valse verklaring",
 "bij alle een onjuist stelling",
 "bij alle een groot fout",
 "bij alle een ernstig fout",
 "bij alle een enorm misverstand",
 "bij alle een belangrijk fout",
 "bij alle een ernstig fout",
 "bij alle een duidelijk fout",
 "bij alle een eenvoudig fout",
 "bij alle een blatant fout",
 "bij alle een significant fout",
 "bij alle een groot fout",
 "bij alle een belangrijk blunder",
 "bij alle een blatant misinterpretatie",
 "bij alle een belangrijk misjudgment",
 "bij alle een ernstig misinterpretatie",
 "bij alle een belangrijk misverstand",
 "bij alle een ernstig blunder",
 "bij alle een groot misinterpretatie",
 "bij alle een compleet misverstand",
 "bij alle een enorm fout",
 "bij alle een grave misinterpretatie",
 "bij alle een significant misinterpretatie",
 "definitely thats not true",
 "definitely thats not correct",
 "definitely thats a mistake",
 "definitely thats an error",
 "definitely thats a misunderstanding",
 "definitely thats a blunder",
 "definitely thats a false statement",
 "definitely thats an incorrect assertion",
 "definitely thats a gross mistake",
 "definitely thats a serious error",
 "definitely thats a huge misunderstanding",
 "definitely thats a major mistake",
 "definitely thats a grave error",
 "definitely thats a clear mistake",
 "definitely thats a simple error",
 "definitely thats a blatant error",
 "definitely thats a significant mistake",
 "definitely thats a gross error",
 "definitely thats a major blunder",
 "definitely thats a blatant misinterpretation",
 "definitely thats a major misjudgment",
 "definitely thats a serious misinterpretation",
 "definitely thats a major misunderstanding",
 "definitely thats a serious blunder",
 "definitely thats a gross misinterpretation",
 "definitely thats a complete misunderstanding",
 "definitely thats a huge mistake",
 "definitely thats a grave misinterpretation",
 "definitely thats a significant misinterpretation",
 "zeker dat niet waar",
 "zeker dat niet correct",
 "zeker dat een fout",
 "zeker dat een fout",
 "zeker dat een misverstand",
 "zeker dat een blunder",
 "zeker dat een valse stelling",
 "zeker dat een onjuiste stelling",
 "zeker dat een grove fout",
 "zeker dat een ernstige fout",
 "zeker dat een grote misverstand",
 "zeker dat een belangrijke fout",
 "zeker dat een ernstige fout",
 "zeker dat een heldere fout",
 "zeker dat een eenvoudige fout",
 "zeker dat een flagrante fout",
 "zeker dat een significante fout",
 "zeker dat een grove fout",
 "zeker dat een belangrijke blunder",
 "zeker dat een flagrante misinterpretatie",
 "zeker dat een belangrijke foute inschatting",
 "zeker dat een ernstige misinterpretatie",
 "zeker dat een belangrijke misverstand",
 "zeker dat een ernstige blunder",
 "zeker dat een grove misinterpretatie",
 "zeker dat een volledige misverstand",
 "zeker dat een grote fout",
 "zeker dat een grave misinterpretatie",
 "zeker dat een significante misinterpretatie",
 "under thats not true",
 "under thats not correct",
 "under thats a mistake",
 "under thats an error",
 "under thats a misunderstanding",
 "under thats a blunder",
 "under thats a false statement",
 "under thats an incorrect assertion",
 "under thats a gross mistake",
 "under thats a serious error",
 "under thats a huge misunderstanding",
 "under thats a major mistake",
 "under thats a grave error",
 "under thats a clear mistake",
 "under thats a simple error",
 "under thats a blatant error",
 "under thats a significant mistake",
 "under thats a gross error",
 "under thats a major blunder",
 "under thats a blatant misinterpretation",
 "under thats a major misjudgment",
 "under thats a serious misinterpretation",
 "under thats a major misunderstanding",
 "under thats a serious blunder",
 "under thats a gross misinterpretation",
 "under thats a complete misunderstanding",
 "under thats a huge mistake",
 "under thats a grave misinterpretation",
 "under thats a significant misinterpretation",
 "onder dat niet waar",
 "onder dat niet correct",
 "onder dat een fout",
 "onder dat een fout",
 "onder dat een misverstand",
 "onder dat een blunder",
 "onder dat een onjuiste verklaring",
 "onder dat een incorrecte stelling",
 "onder dat een bruto fout",
 "onder dat een ernstige fout",
 "onder dat een enorme misverstand",
 "onder dat een grote fout",
 "onder dat een ernstige fout",
 "onder dat een duidelijke fout",
 "onder dat een simpele fout",
 "onder dat een opvallende fout",
 "onder dat een significante fout",
 "onder dat een bruto fout",
 "onder dat een grote blunder",
 "onder dat een opvallende misinterpretatie",
 "onder dat een grote misjudgment",
 "onder dat een ernstige misinterpretatie",
 "onder dat een grote misverstand",
 "onder dat een ernstige blunder",
 "onder dat een bruto misinterpretatie",
 "onder dat een volledige misverstand",
 "onder dat een enorme fout",
 "onder dat een ernstige misinterpretatie",
 "onder dat een significante misinterpretatie",
 "decline thats not true",
 "decline thats not correct",
 "decline thats a mistake",
 "decline thats an error",
 "decline thats a misunderstanding",
 "decline thats a blunder",
 "decline thats a false statement",
 "decline thats an incorrect assertion",
 "decline thats a gross mistake",
 "decline thats a serious error",
 "decline thats a huge misunderstanding",
 "decline thats a major mistake",
 "decline thats a grave error",
 "decline thats a clear mistake",
 "decline thats a simple error",
 "decline thats a blatant error",
 "decline thats a significant mistake",
 "decline thats a gross error",
 "decline thats a major blunder",
 "decline thats a blatant misinterpretation",
 "decline thats a major misjudgment",
 "decline thats a serious misinterpretation",
 "decline thats a major misunderstanding",
 "decline thats a serious blunder",
 "decline thats a gross misinterpretation",
 "decline thats a complete misunderstanding",
 "decline thats a huge mistake",
 "decline thats a grave misinterpretation",
 "decline thats a significant misinterpretation",
 "weiger dat niet waar",
 "weiger dat niet correct",
 "weiger dat een fout",
 "weiger dat een fout",
 "weiger dat een misverstand",
 "weiger dat een blunder",
 "weiger dat een valse stelling",
 "weiger dat een onjuist bewering",
 "weiger dat een grove fout",
 "weiger dat een ernstige fout",
 "weiger dat een grote misverstand",
 "weiger dat een belangrijke fout",
 "weiger dat een ernstige fout",
 "weiger dat een heldere fout",
 "weiger dat een simpele fout",
 "weiger dat een flagrante fout",
 "weiger dat een significante fout",
 "weiger dat een grove fout",
 "weiger dat een belangrijke blunder",
 "weiger dat een flagrante misinterpretatie",
 "weiger dat een belangrijke misjudgment",
 "weiger dat een ernstige misinterpretatie",
 "weiger dat een belangrijke misverstand",
 "weiger dat een ernstige blunder",
 "weiger dat een grove misinterpretatie",
 "weiger dat een complete misverstand",
 "weiger dat een grote fout",
 "weiger dat een grave misinterpretatie",
 "weiger dat een significant misinterpretatie",
 "refuse thats not true",
 "refuse thats not correct",
 "refuse thats a mistake",
 "refuse thats an error",
 "refuse thats a misunderstanding",
 "refuse thats a blunder",
 "refuse thats a false statement",
 "refuse thats an incorrect assertion",
 "refuse thats a gross mistake",
 "refuse thats a serious error",
 "refuse thats a huge misunderstanding",
 "refuse thats a major mistake",
 "refuse thats a grave error",
 "refuse thats a clear mistake",
 "refuse thats a simple error",
 "refuse thats a blatant error",
 "refuse thats a significant mistake",
 "refuse thats a gross error",
 "refuse thats a major blunder",
 "refuse thats a blatant misinterpretation",
 "refuse thats a major misjudgment",
 "refuse thats a serious misinterpretation",
 "refuse thats a major misunderstanding",
 "refuse thats a serious blunder",
 "refuse thats a gross misinterpretation",
 "refuse thats a complete misunderstanding",
 "refuse thats a huge mistake",
 "refuse thats a grave misinterpretation",
 "refuse thats a significant misinterpretation",
 "weiger dat niet waar",
 "weiger dat niet juist",
 "weiger dat een fout",
 "weiger dat een fout",
 "weiger dat een misverstand",
 "weiger dat een blunder",
 "weiger dat een onjuiste stelling",
 "weiger dat een onjuiste stelling",
 "weiger dat een grove fout",
 "weiger dat een ernstige fout",
 "weiger dat een enorme misverstand",
 "weiger dat een belangrijke fout",
 "weiger dat een ernstige fout",
 "weiger dat een duidelijke fout",
 "weiger dat een eenvoudige fout",
 "weiger dat een flagrante fout",
 "weiger dat een significante fout",
 "weiger dat een grove fout",
 "weiger dat een belangrijke blunder",
 "weiger dat een flagrante misinterpretatie",
 "weiger dat een belangrijke onjuiste beoordeling",
 "weiger dat een ernstige misinterpretatie",
 "weiger dat een belangrijke misverstand",
 "weiger dat een ernstige blunder",
 "weiger dat een grove misinterpretatie",
 "weiger dat een volledig misverstand",
 "weiger dat een enorme fout",
 "weiger dat een ernstige misinterpretatie",
 "weiger dat een significante misinterpretatie",
 "disagree thats not true",
 "disagree thats not correct",
 "disagree thats a mistake",
 "disagree thats an error",
 "disagree thats a misunderstanding",
 "disagree thats a blunder",
 "disagree thats a false statement",
 "disagree thats an incorrect assertion",
 "disagree thats a gross mistake",
 "disagree thats a serious error",
 "disagree thats a huge misunderstanding",
 "disagree thats a major mistake",
 "disagree thats a grave error",
 "disagree thats a clear mistake",
 "disagree thats a simple error",
 "disagree thats a blatant error",
 "disagree thats a significant mistake",
 "disagree thats a gross error",
 "disagree thats a major blunder",
 "disagree thats a blatant misinterpretation",
 "disagree thats a major misjudgment",
 "disagree thats a serious misinterpretation",
 "disagree thats a major misunderstanding",
 "disagree thats a serious blunder",
 "disagree thats a gross misinterpretation",
 "disagree thats a complete misunderstanding",
 "disagree thats a huge mistake",
 "disagree thats a grave misinterpretation",
 "disagree thats a significant misinterpretation",
 "nee dat niet waar",
 "nee dat niet juist",
 "nee dat een fout",
 "nee dat een fout",
 "nee dat een misverstand",
 "nee dat een blunder",
 "nee dat een onjuist bewering",
 "nee dat een onjuist assertie",
 "nee dat een grote fout",
 "nee dat een ernstige fout",
 "nee dat een enorme misverstand",
 "nee dat een grote fout",
 "nee dat een serieuze fout",
 "nee dat een heldere fout",
 "nee dat een eenvoudige fout",
 "nee dat een flagrante fout",
 "nee dat een significante fout",
 "nee dat een grote fout",
 "nee dat een grote blunder",
 "nee dat een flagrante misinterpretatie",
 "nee dat een grote misjudgement",
 "nee dat een serieuze misinterpretatie",
 "nee dat een grote misverstand",
 "nee dat een serieuze blunder",
 "nee dat een grote misinterpretatie",
 "nee dat een complete misverstand",
 "nee dat een enorme fout",
 "nee dat een grave misinterpretatie",
 "nee dat een significant misinterpretatie",
 "sorry thats not true",
 "sorry thats not correct",
 "sorry thats a mistake",
 "sorry thats an error",
 "sorry thats a misunderstanding",
 "sorry thats a blunder",
 "sorry thats a false statement",
 "sorry thats an incorrect assertion",
 "sorry thats a gross mistake",
 "sorry thats a serious error",
 "sorry thats a huge misunderstanding",
 "sorry thats a major mistake",
 "sorry thats a grave error",
 "sorry thats a clear mistake",
 "sorry thats a simple error",
 "sorry thats a blatant error",
 "sorry thats a significant mistake",
 "sorry thats a gross error",
 "sorry thats a major blunder",
 "sorry thats a blatant misinterpretation",
 "sorry thats a major misjudgment",
 "sorry thats a serious misinterpretation",
 "sorry thats a major misunderstanding",
 "sorry thats a serious blunder",
 "sorry thats a gross misinterpretation",
 "sorry thats a complete misunderstanding",
 "sorry thats a huge mistake",
 "sorry thats a grave misinterpretation",
 "sorry thats a significant misinterpretation",
 "sorry dat niet waar",
 "sorry dat niet juist",
 "sorry dat een fout",
 "sorry dat een fout",
 "sorry dat een misverstand",
 "sorry dat een blunder",
 "sorry dat een valse verklaring",
 "sorry dat een onjuiste beweering",
 "sorry dat een grove fout",
 "sorry dat een ernstige fout",
 "sorry dat een enorm misverstand",
 "sorry dat een belangrijke fout",
 "sorry dat een serieuze fout",
 "sorry dat een ernstige fout",
 "sorry dat een duidelijke fout",
 "sorry dat een eenvoudige fout",
 "sorry dat een flagrante fout",
 "sorry dat een significante fout",
 "sorry dat een grove fout",
 "sorry dat een belangrijke blunder",
 "sorry dat een flagrante misinterpretatie",
 "sorry dat een belangrijke onjuiste beoordeling",
 "sorry dat een serieuze misinterpretatie",
 "sorry dat een belangrijke misverstand",
 "sorry dat een serieuze blunder",
 "sorry dat een grove misinterpretatie",
 "sorry dat een volledige misverstand",
 "sorry dat een enorme fout",
 "sorry dat een ernstige misinterpretatie",
 "sorry dat een significante misinterpretatie",
 "unfortunately thats not true",
 "unfortunately thats not correct",
 "unfortunately thats a mistake",
 "unfortunately thats an error",
 "unfortunately thats a misunderstanding",
 "unfortunately thats a blunder",
 "unfortunately thats a false statement",
 "unfortunately thats an incorrect assertion",
 "unfortunately thats a gross mistake",
 "unfortunately thats a serious error",
 "unfortunately thats a huge misunderstanding",
 "unfortunately thats a major mistake",
 "unfortunately thats a grave error",
 "unfortunately thats a clear mistake",
 "unfortunately thats a simple error",
 "unfortunately thats a blatant error",
 "unfortunately thats a significant mistake",
 "unfortunately thats a gross error",
 "unfortunately thats a major blunder",
 "unfortunately thats a blatant misinterpretation",
 "unfortunately thats a major misjudgment",
 "unfortunately thats a serious misinterpretation",
 "unfortunately thats a major misunderstanding",
 "unfortunately thats a serious blunder",
 "unfortunately thats a gross misinterpretation",
 "unfortunately thats a complete misunderstanding",
 "unfortunately thats a huge mistake",
 "unfortunately thats a grave misinterpretation",
 "unfortunately thats a significant misinterpretation",
 "helaas dat niet waar",
 "helaas dat niet correct",
 "helaas dat een fout",
 "helaas dat een fout",
 "helaas dat een misverstand",
 "helaas dat een blunder",
 "helaas dat een valse verklaring",
 "helaas dat een onjuiste stelling",
 "helaas dat een brutale fout",
 "helaas dat een ernstige fout",
 "helaas dat een enorme misverstand",
 "helaas dat een grote fout",
 "helaas dat een grave fout",
 "helaas dat een duidelijke fout",
 "helaas dat een simpele fout",
 "helaas dat een blatante fout",
 "helaas dat een significante fout",
 "helaas dat een brutale fout",
 "helaas dat een grote blunder",
 "helaas dat een blatante misinterpretatie",
 "helaas dat een grote misjudgement",
 "helaas dat een ernstige misinterpretatie",
 "helaas dat een grote misverstand",
 "helaas dat een ernstige blunder",
 "helaas dat een brutale misinterpretatie",
 "helaas dat een volledige misverstand",
 "helaas dat een enorme fout",
 "helaas dat een grave misinterpretatie",
 "helaas dat een significante misinterpretatie",
 "sadly thats not true",
 "sadly thats not correct",
 "sadly thats a mistake",
 "sadly thats an error",
 "sadly thats a misunderstanding",
 "sadly thats a blunder",
 "sadly thats a false statement",
 "sadly thats an incorrect assertion",
 "sadly thats a gross mistake",
 "sadly thats a serious error",
 "sadly thats a huge misunderstanding",
 "sadly thats a major mistake",
 "sadly thats a grave error",
 "sadly thats a clear mistake",
 "sadly thats a simple error",
 "sadly thats a blatant error",
 "sadly thats a significant mistake",
 "sadly thats a gross error",
 "sadly thats a major blunder",
 "sadly thats a blatant misinterpretation",
 "sadly thats a major misjudgment",
 "sadly thats a serious misinterpretation",
 "sadly thats a major misunderstanding",
 "sadly thats a serious blunder",
 "sadly thats a gross misinterpretation",
 "sadly thats a complete misunderstanding",
 "sadly thats a huge mistake",
 "sadly thats a grave misinterpretation",
 "sadly thats a significant misinterpretation",
 "helaas dat niet waar",
 "helaas dat niet correct",
 "helaas dat een fout",
 "helaas dat een fout",
 "helaas dat een misverstand",
 "helaas dat een blunder",
 "helaas dat een valse verklaring",
 "helaas dat een onjuist stelling",
 "helaas dat een groot fout",
 "helaas dat een ernstige fout",
 "helaas dat een enorm misverstand",
 "helaas dat een belangrijk fout",
 "helaas dat een ernstig fout",
 "helaas dat een duidelijk fout",
 "helaas dat een eenvoudig fout",
 "helaas dat een overduidelijk fout",
 "helaas dat een significant fout",
 "helaas dat een groot fout",
 "helaas dat een belangrijk blunder",
 "helaas dat een overduidelijk misinterpretatie",
 "helaas dat een belangrijk fout",
 "helaas dat een ernstig misinterpretatie",
 "helaas dat een belangrijk misverstand",
 "helaas dat een ernstig blunder",
 "helaas dat een groot misinterpretatie",
 "helaas dat een volledig misverstand",
 "helaas dat een enorm fout",
 "helaas dat een ernstig misinterpretatie",
 "helaas dat een significant misinterpretatie",
 "sadly thats not true",
 "sadly thats not correct",
 "sadly thats a mistake",
 "sadly thats an error",
 "sadly thats a misunderstanding",
 "sadly thats a blunder",
 "sadly thats a false statement",
 "sadly thats an incorrect assertion",
 "sadly thats a gross mistake",
 "sadly thats a serious error",
 "sadly thats a huge misunderstanding",
 "sadly thats a major mistake",
 "sadly thats a grave error",
 "sadly thats a clear mistake",
 "sadly thats a simple error",
 "sadly thats a blatant error",
 "sadly thats a significant mistake",
 "sadly thats a gross error",
 "sadly thats a major blunder",
 "sadly thats a blatant misinterpretation",
 "sadly thats a major misjudgment",
 "sadly thats a serious misinterpretation",
 "sadly thats a major misunderstanding",
 "sadly thats a serious blunder",
 "sadly thats a gross misinterpretation",
 "sadly thats a complete misunderstanding",
 "sadly thats a huge mistake",
 "sadly thats a grave misinterpretation",
 "sadly thats a significant misinterpretation",
 "sadly",
 "not going to",
 "don't",
 "do not",
 "sorry",
 "no",
 "not",
 "prefer not",
 "definitely not",
 "never",
 "unfortunately",
 "sadly",
 "regrettably",
 "not really",
 "not particularly",
 "not good",
 "not nice",
 "not fantastic",
 "not great",
 "not top",
 "not brilliant",
 "not super",
 "not wonderful",
 "not splendid",
 "not perfect",
 "not outstanding",
 "not excellent",
 "not phenomenal",
 "not fabulous",
 "not obvious",
 "not without doubt",
 "not wholeheartedly",
 "not with pleasure",
 "definitely",
 "not",
 "certainly not",
 "absolutely no",
 "really not",
 "not good",
 "not okay",
 "not happy",
 "not happy with",
 "disappointing",
 "bad",
 "helaas dat niet waar",
 "helaas dat niet juist",
 "helaas dat een fout",
 "helaas dat een fout",
 "helaas dat een misverstand",
 "helaas dat een blunder",
 "helaas dat een valse verklaring",
 "helaas dat een onjuist stelling",
 "helaas dat een grote fout",
 "helaas dat een ernstige fout",
 "helaas dat een enorme misverstand",
 "helaas dat een grote fout",
 "helaas dat een gravier fout",
 "helaas dat een heldere fout",
 "helaas dat een eenvoudige fout",
 "helaas dat een flagrante fout",
 "helaas dat een significante fout",
 "helaas dat een flagrante misinterpretatie",
 "helaas dat een belangrijke misjudgment",
 "helaas dat een ernstige misinterpretatie",
 "helaas dat een belangrijke misverstand",
 "helaas dat een ernstige blunder",
 "helaas dat een flagrante misinterpretatie",
 "helaas dat een volledige misverstand",
 "helaas dat een enorme fout",
 "helaas dat een gravier misinterpretatie",
 "helaas dat een significante misinterpretatie",
 "helaas",
 "niet gaan om",
 "doen",
 "doen niet",
 "sorry",
 "nee",
 "niet",
 "liever niet",
 "zeker niet",
 "nooit",
 "helaas",
 "spijtig",
 "jammer genoeg",
 "echt niet",
 "zeker niet",
 "absoluut nee",
 "echt niet",
 "niet goed",
 "niet okΘ",
 "niet blij",
 "niet blij met",
 "teleurstellend",
 "slecht"
]


In [15]:
for i in texts:
  analyze(i)

word_predictions: [('no', 'O')] 

word_predictions: [('nope', 'O')] 

word_predictions: [('not', 'O')] 

word_predictions: [('won', 'O'), ("'", 'O'), ('t', 'O')] 

word_predictions: [('will', 'O'), ('not', 'O')] 

word_predictions: [('can', 'O'), ("'", 'O'), ('t', 'O')] 

word_predictions: [('cannot', 'O')] 

word_predictions: [('negative', 'O')] 

word_predictions: [('nay', 'O')] 

word_predictions: [('absolutely', 'O'), ('not', 'O')] 

word_predictions: [('certainly', 'O'), ('not', 'O')] 

word_predictions: [('never', 'O')] 

word_predictions: [('no', 'O'), ('way', 'O')] 

word_predictions: [('not', 'O'), ('at', 'O'), ('all', 'O')] 

word_predictions: [('definitely', 'O'), ('not', 'O')] 

word_predictions: [('under', 'O'), ('no', 'O'), ('circumstances', 'O')] 

word_predictions: [('decline', 'O')] 

word_predictions: [('refuse', 'O')] 

word_predictions: [('disagree', 'O')] 

word_predictions: [('not', 'O'), ('going', 'O'), ('to', 'O')] 

word_predictions: [('don', 'O'), ("'", 'O'), 

In [16]:
Name = "Aagtje"
texts = [
 "My name is",
 "They call me",
 "call me",
 "name is",
 "ze noemen me",
 "mijn naam",
 "noem me",
 "naam is",
 "I am called",
 "I'm known as",
 "you can call me",
 "naam",
 "this is",
 "here is",
 "I go by",
 "people know me as",
 "ik heet",
 "men noemt mij",
 "u kunt me noemen",
 "dit is",
 "hier is",
 "ik ga als",
 "mensen kennen mij als",
 "meet",
 "say hi to",
 "greet",
 "introducing",
 "ik ben",
 "dit is je jongen",
 "dit is je meisje",
 "ontmoet",
 "zeg hallo tegen",
 "groet",
 "ik presenteer",
 "ja da's mijn naam",
 "ja dat is mijn naam",
 "ja dat was mijn naam",
 "no youre mistaken my name is",
 "actually my name is",
 "im afraid thats incorrect my name is",
 "i am afraid thats incorrect my name is",
 "no its",
 "thats not right my name is",
 "nope im",
 "nope i am",
 "not at all im",
 "not at all i am",
 "no youve got it wrong my name is",
 "no you have got it wrong my name is",
 "im sorry but my name is actually",
 "i am sorry but my name is actually",
 "no thats a mistake my name is",
 "not quite my name is",
 "im afraid thats not right its",
 "i am afraid thats not right its",
 "no youve mixed it up im",
 "no you have mixed it up i am",
 "no thats not true my name is",
 "no youve got the wrong person im",
 "no you have got the wrong person i am",
 "no youve mixed me up with someone else im",
 "no you have mixed me up with someone else i am",
 "not really my name is",
 "definitely not my name is",
 "nee je vergist mijn naam is",
 "eigenlijk is mijn naam",
 "ik ben bang dat niet klopt mijn naam is",
 "nee het is",
 "dat is niet juist mijn naam",
 "nee hoor ik ben",
 "helemaal niet ik ben",
 "zeker niet mijn naam is",
 "nee je hebt het mis mijn naam is",
 "sorry maar mijn naam is eigenlijk",
 "nee dat is een vergissing mijn naam",
 "niet helemaal mijn naam is",
 "ik ben bang dat niet klopt het is",
 "nee je hebt het door elkaar gehaald ik ben",
 "nee dat is niet waar mijn naam",
 "nee je hebt de verkeerde persoon ik ben",
 "nee je hebt me met iemand anders verward ik ben",
 "niet echt mijn naam is"
]
tt = []

# Loop through each sentence in texts and append the full sentence to tt
for sentence in texts:
    tt.append(sentence + " " + Name)

# Output the result?
# print(tt)
texts = tt

In [17]:
for i in texts:
  analyze(i)

word_predictions: [('my', 'O'), ('name', 'O'), ('is', 'O'), ('aagtje', 'B-PER')] 

word_predictions: [('they', 'O'), ('call', 'O'), ('me', 'O'), ('aagtje', 'O')] 

word_predictions: [('call', 'O'), ('me', 'O'), ('aagtje', 'O')] 

word_predictions: [('name', 'O'), ('is', 'O'), ('aagtje', 'B-PER')] 

word_predictions: [('ze', 'O'), ('noemen', 'O'), ('me', 'O'), ('aagtje', 'O')] 

word_predictions: [('mijn', 'O'), ('naam', 'O'), ('aagtje', 'B-PER')] 

word_predictions: [('noem', 'O'), ('me', 'O'), ('aagtje', 'O')] 

word_predictions: [('naam', 'O'), ('is', 'O'), ('aagtje', 'B-PER')] 

word_predictions: [('i', 'O'), ('am', 'O'), ('called', 'O'), ('aagtje', 'B-PER')] 

word_predictions: [('i', 'O'), ("'", 'O'), ('m', 'O'), ('known', 'O'), ('as', 'O'), ('aagtje', 'B-PER')] 

word_predictions: [('you', 'O'), ('can', 'O'), ('call', 'O'), ('me', 'O'), ('aagtje', 'B-PER')] 

word_predictions: [('naam', 'O'), ('aagtje', 'B-PER')] 

word_predictions: [('this', 'O'), ('is', 'O'), ('aagtje', 'O')] 


In [ ]:
# # prompt: mount google drive

# from google.colab import drive
# drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# !pip install pandas openpyxl

In [ ]:
# import pandas as pd

# # Load the Excel file
# file_path = '/content/drive/MyDrive/dutch name_list/Surnames.xlsx'
# df = pd.read_excel(file_path)

# # Extract the names from the first column
# names = df.iloc[:, 0].tolist()
# first_20_names = names[:20]

# print(first_20_names)
# # first_20_names = ["ram","susan dulal","rakshya bhetwal","sohan","rikesh"]

# for name in first_20_names:
#     text=f"my name is {name.lower()}. I live in udaypur."
#     analyze(text)

In [ ]:
# import pandas as pd

# # Load the Excel file
# file_path = '/content/drive/MyDrive/dutch name_list/data-1693309667413 firstname and lastnames.xlsx'
# df = pd.read_excel(file_path)

# # Extract the names from the first column
# first_names = df[df['category'] == 'Firstname']['name'].tolist()
# first_20_names = first_names[:20]

# print(first_20_names)
# # first_20_names = ["ram","susan dulal","rakshya bhetwal","sohan","rikesh"]

# for name in first_20_names:
#     text=f"my name is {name.lower()}. I live in udaypur."
#     # text=f"They call me {name}. I live in udaypur."
#     analyze(text)